In [174]:
import numpy as np
from itertools import product
from typing import List

In [172]:
class BaseClass:
    
    eyes = ['left', 'right']
    orientations = [1, 2]

# Classes for neurons

In [173]:
class SensoryNeuron(BaseClass):
    
    def __init__(
        self,
        eye,
        orientation,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation,
        init_response,
        init_habituation
    ):
        self.eye = eye
        self.orientation = orientation
        self.alpha = alpha
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.tau_habituation = tau_habituation
        self.response = init_response
        self.habituation = init_habituation
        
    def one_timestep(
        self,
        suppressive_drive,
        dt
    ):
        _change_in_habituation = self._calculate_change_in_habituation(dt)
        _change_in_response = self._calculate_change_in_response(suppressive_drive, dt)
        self.response += _change_in_response
        self.habituation += _change_in_habituation
        
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        sensory_input,
        snapshot
    ):
        attention_response = getattr(snapshot, f'attention_{self.orientation}')
        opponency_response = np.sum(
            [getattr(snapshot, 'opponency_{}_{}'.format(self.eye, orientation)) for orientation in self.orientations]
        )
        self._excitatory_drive = rectification(
            sensory_input ** self.n - self.weight_opponency * opponency_response
        ) * rectification(
            1 + self.weight_attention * attention_response
        )
    
    def _calculate_change_in_response(
        self,
        suppressive_drive
    ):
        change_in_response = (
            - self.response + 
            (self.alpha * self.excitatory_drive) / 
            (suppressive_drive + self.habituation ** self.n + self.sigma ** self.n)
        ) * (dt / self.tau_response)
        return change_in_response
        
    def _calculate_change_in_habituation(
        self
    ):
        change_in_habituation = (
            self.habituation + 
            self.weight_habituation * self.response
        ) * (dt / self.tau_habituation)
        return change_in_habituation

In [91]:
class SummationNeuron(BaseClass):
    
    def __init__(
        self,
        orientation,
        sigma,
        n,
        weight_habituation,
        tau_response,
        tau_habituation,
        init_response,
        init_habituation
    ):
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.tau_habituation = tau_habituation
        self.init_response = init_response
        self.init_habituation = init_habituation
        
    def one_timestep(
        self,
        dt
    ):
        _change_in_habituation = self._calculate_change_in_habituation(dt)
        _change_in_response = self._calculate_change_in_response(dt)
        self.response += _change_in_response
        self.habituation += _change_in_habituation
            
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        snapshot
    ):
        response_left = getattr(snapshot, f'sensory_left_{self.orientation}')
        response_right = getattr(snapshot, f'sensory_right_{self.orientation}')
        self._excitatory_drive = (response_left + response_right) ** self.n
        
    @property
    def suppressive_drive(
        self
    ):
        return self._excitatory_drive
    
    #option: def one_timestep
    def _calculate_change_in_response(
        self,
        dt
    ):
        change_in_response = (
            - self.response + self.excitatory_drive / 
            (self.suppressive_drive + self.habituation ** self.n + self.sigma ** self.n)
        ) * (dt / self.tau_response)
        return change_in_response
    
    def _calculate_change_in_habituation(
        self,
        dt
    ):
        change_in_habituation = (
            self.habituation + 
            self.weight_habituation * self.response
        ) * (dt / self.tau_habituation)
        return change_in_habituation

In [98]:
class OpponencyNeuron(BaseClass):
    
    def __init__(
        self,
        eye,
        orientation,
        sigma,
        n,
        tau_response,
        init_response
    ):
        self.eye = eye
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.tau_response = tau_response
        self.init_response = init_response
        
    def one_timestep(
        self,
        suppressive_drive,
        dt
    ):
        _change_in_response = self.calculate_change_in_response(suppressive_drive, dt)
        self.response += _change_in_response
        
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        snapshot
    ):
        
        self._excitatory_drive = rectification(response_same_eye - response_other_eye) ** self.n
        
    def calculate_change_in_response(
        self,
        suppressive_drive,
        dt
    ):
        change_in_response = (
            - self.response +
            self.excitatory_drive /
            (suppressive_drive + self.sigma ** self.n)
        ) * (dt / self.tau_response)
        return change_in_response

In [98]:
class AttentionNeuron(BaseClass):
    
    def __init__(
        self,
        orientation,
        sigma,
        n,
        tau_response,
        init_response
    ):
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.tau_response = tau_response
        self.response = init_response
        
    def compute_excitatory_drive(
        self,
        snapshot
    ):
        response_same_orientation = getattr(
            snapshot, f'summation_{self.orientation}'
        )
        response_other_orientation = getattr(
            snapshot, f'summation_{sum(self.orientations) - self.orientation}'
        )
        self._excitatory_drive = (response_same_orientation - response_other_orientation) ** self.n
    
    def one_timestep(
        self,
        suppressive_drive,
        dt
    ):
        _change_in_response = self.calculate_change_in_response(suppressive_drive, dt)
        self.response += _change_in_response
        
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
        
    def calculate_change_in_response(
        self,
        suppressive_drive,
        dt
    ):
        change_in_response = (
            - self.response +
            self.excitatory_drive /
            (suppressive_drive + self.sigma ** self.n)
        ) * (dt / self.tau_response)
        return change_in_response

# Classes for populations

In [79]:
class SensoryPopulation(BaseClass):
    
    def __init__(
        self,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for eye, orientation in product(self.eyes, self.orientations):
            key = eye + '_' + orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:    
                _initial_habituation = np.random.rand()
            self.neurons[key] = SensoryNeuron(
                eye=eye,
                orientation=orientation,
                alpha=alpha,
                sigma=sigma,
                n=n,
                weight_opponency=weight_opponency,
                weight_attention=weight_attention,
                weight_habituation=weight_habituation,
                tau_response=tau_response,
                tau_habituation=tau_habituation,
                initial_response=initial_response,
                initial_habituation=initial_habituation
            )
            
    def compute_excitatory_drive(
        self,
        sensory_input,
        snap
    ):
        for neuron in neurons.values():
            neuron.compute_excitatory_drive(sensory_input, snap)
        
    def one_timestep(
        self,
        sensory_input,
        snap
    ):
        for neuron in neurons.values():
            neuron.one_timestep(self.suppressive_drive, dt)
        
    @property
    def suppressive_drive(self):
        return sum([neuron.excitatory_drive for neuron in self.neurons.values()])

In [92]:
class SummationPopulation:
    
    orientations = [1, 2]
    
    def __init__(
        self,
        sigma,
        n,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for orientation in self.orientations:
            key = orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:    
                _initial_habituation = np.random.rand()
            self.neurons[key] = SummationNeuron(
                orientation=orientation,
                sigma=sigma,
                n=n,
                weight_habituation=weight_habituation,
                tau_response=tau_response,
                tau_habituation=tau_habituation,
                initial_response=initial_response,
                initial_habituation=initial_habituation
            )
            
    def compute_excitatory_drive(
        self,
        snapshot
    ):
        for neuron in self.neurons.values():
            neuron.compute_excitatory_drive(snapshot)
            
    def one_timestep(
        self
    ):
        for orientation in self.orientations:
            self.neurons[orientation].one_timestep()

In [100]:
class OpponencyPopulation:
    
    eyes = ['left', 'right']
    orientations = [1, 2]
    
    def __init__(
        self,
        sigma,
        n,
        tau_response,
        initial_response=None,
    ):
        for eye, orientation in product(self.eyes, self.orientations):
            key = '_'.join([eye, orientation])
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            self.neurons[key] = OpponencyNeuron(
                eye=eye,
                orientation=orientation,
                sigma=sigma,
                n=n,
                tau_response=tau_response,
                initial_response=initial_response,
            )
            
    def compute_excitatory_drives(
        self,
        snapshot
    ):
        for neuron in self.neurons.values():
            neuron.compute_excitatory_drive(snapshot)
        
    def one_timestep(
        self,
        dt
    ):
        suppressive_drives = self.suppressive_drives
        for eye in self.eyes:
            suppressive_drive_eye = suppressive_drives[eye]
            for orientation in self.orientations:
                key = '_'.join([eye, orientation])
                self.neurons[orientation].one_timestep(suppressive_drive_eye, dt)
                
    @property
    def suppressive_drives(self):
        drives = {}
        for eye in self.eyes:
            drives[eye] = sum([
                neurons['_'.join([eye, orientation])].excitatory_drive for orientation in self.orientations])
        return drives

In [81]:
class AttentionPopulation(BaseClass):
    
    def __init__(
        self,
        sigma,
        n,
        tau_response,
        initial_response=None,
    ):
        self.neurons = {}
        for orientation in self.orientations:
            key = orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                print(f'WARNING: Taking random start value for attention neuron for orientation {key}')
                _initial_response = np.random.rand()
            self.neurons[key] = AttentionNeuron(
                orientation=orientation,
                sigma=sigma,
                n=n,
                tau_response=tau_response,
                initial_response=_initial_response
            )
            
    def compute_excitatory_drives(
        self,
        snapshot
    ):
        for neuron in self.neurons.values():
            neuron.compute_excitatory_drive(snapshot)
        
    def one_timestep(
        self,
        dt
    ):
        for neuron in self.neurons.values():
            neuron.one_timestep(self.suppressive_drives, dt)
                
    @property
    def suppressive_drives(self):
        return np.sum(
            [rectification(neuron.excitatory_drive) for neuron in self.neurons.values()]
        )

# Network level

In [130]:
class Network(BaseClass):
    
    def __init__(
        self, 
        dt, 
        attention_populations_arguments = {},
        *args, 
        **kwargs
    ):
        self.init_populations(attention_populations_arguments, *args, **kwargs)
        self.take_snap()
        
    def take_snap(self):
        self._snap = Snapshot(
            sensory_left_1 = self.populations['sensory'].neurons['left_1'].response,
            sensory_left_2 = self.populations['sensory'].neurons['left_2'].response,
            sensory_right_1 = self.populations['sensory'].neurons['right_1'].response,
            sensory_right_2 = self.populations['sensory'].neurons['right_2'].response,
            summary_1 = self.populations['summary'].neurons['1'].response,
            summary_2 = self.populations['sensory'].neurons['2'].response,
            opponency_left_1 = self.populations['opponency'].neurons['left_1'].response,
            opponency_left_2 = self.populations['opponency'].neurons['left_2'].response,
            opponency_right_1 = self.populations['opponency'].neurons['right_1'].response,
            opponency_right_2 = self.populations['opponency'].neurons['right_2'].response,
            attention_1 = self.populations['attention'].neurons['1'].response,
            attention_2 = self.populations['attention'].neurons['2'].response
        )
        
    def init_populations(self, attention_populations_arguments = {}, *args, **kwargs):
        self.populations = {}
        self.populations['sensory'] = SensoryPopulation()
        self.populations['summation'] = SummationPopulation()
        self.populations['attention'] = AttentionPopulation()
        self.populations['opponency'] = OpponencyPopulation()
        
    def simulate(self, sensory_input):
        """
        Expects n_timepoints x 2 sensory_input
        """
        timecourse = Timecourse([self.snap])
        for t in sensory_input.shape[0]:
            self.one_step(sensory_input[t, :])
            timecourse.append(self.snap)
        return timecourse
        
    @property
    def snap(self):
        return self._snap
    
    def one_step(self, sensory_input):
        self.take_snap()
        for population in self.populations:
            population.compute_excitatory_drive(self.snap)
        for population in self.populations:
            population.

snap:  64
dict:  656


# Utilities

In [151]:
@dataclass
class Snapshot:
    
    sensory_left_1: float
    sensory_left_2: float
    sensory_right_1: float
    sensory_right_2: float
    opponency_left_1: float
    opponency_left_2: float
    opponency_right_1: float
    opponency_right_2: float
    summation_1: float
    summation_2: float
    attention_1: float
    attention_2: float        

In [118]:
@dataclass
class Timecourse:
    
    snapshots: List[Snapshot]
        
    def append(self, snap):
        self.snapshots.append(snap)
        
    @property
    def activation_binocular_1(self):
        return np.array([x['summation'][1] for x in self.snapshots])

In [162]:
dic = {
    'sensory_left_1': 1.,
    'sensory_left_2': 1.,
    'sensory_right_1': 1.,
    'sensory_right_2': 1.,
    'opponency_left_1': 1.,
    'opponency_left_2': 1.,
    'opponency_right_1': 1.,
    'opponency_right_2': 1.,
    'summation_1': 1.,
    'summation_2': 1.,
    'attention_1': 1.,
    'attention_2': 1.
}
snap = Snapshot(*list(dic.values()))
print('snap: ', getsizeof(snap))
print('dict: ', getsizeof(dic))

snap:  64
dict:  656


In [177]:
x = np.random.randn(10)

In [183]:
x[x<0] = 0

In [184]:
x

array([0.        , 0.        , 0.81167054, 0.        , 0.52246469,
       1.58914554, 0.        , 0.        , 0.76364831, 0.        ])

In [182]:
[max(_x,0) for _x in x]

[0,
 0,
 0.8116705379669541,
 0,
 0.5224646925555357,
 1.589145544701398,
 0,
 0,
 0.7636483077082552,
 0]

In [198]:
def get_rectification(smooth=True):
    if smooth:
        def smooth_rectification(x, threshold=.05, slope=30):
            x[x < 0] = 0.
            return x / (1 + np.exp( - slope * (x - threshold)))    
        return smooth_rectification
    else:
        def non_smooth_rectification(x, n=1):
            x[x < 0] = 0.
            return x ** n
        return non_smooth_rectification
rectification = get_rectification()




In [199]:
x = np.random.randn(10) * .001
get_rectification()(x)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.71723889e-04, 6.60831066e-05, 1.94841322e-04,
       0.00000000e+00, 0.00000000e+00])

function y=halfExp_smooth(x)
    thresh=0.05;
    slope=30;

    idx_0 = x<0;
    idx_1 = x>0;

    y = zeros(size(x));
    y(idx_0) = 0;
    y(idx_1) = x(idx_1).*1./(1+exp(-slope*(x(idx_1)-thresh)));
end

# General structure

## Plan für morgen (heute):
 * network
 * rectification
 * Snapshot -> Results